# Excited States via SSVQE

This notebook validates that:
- SSVQE runs for `num_states=k` states
- Works under noise

It prints:
- Lowest $k$ exact eigenvalues of $H$
- SSVQE final energies for states $0, 1, ..., k-1$
- $|ΔE_i|$ versus exact eigenvalue i

In [ ]:
from __future__ import annotations

import numpy as np
import pennylane as qml

from vqe.hamiltonian import build_hamiltonian
from vqe.ssvqe import run_ssvqe

## Configuration

In [ ]:
molecule = "H2"

# k-state target
num_states = 4

# Ansatz/optimizer
ansatz_name = "StronglyEntanglingLayers"
optimizer_name = "Adam"
steps = 250
stepsize = 0.2
seed = 0

# SSVQE weights (None => [1,2,3,...])
weights = None

# Noise knobs (leave at 0 for noiseless section)
depolarizing_prob = 0.00
amplitude_damping_prob = 0.00

## Build Hamiltonian + exact spectrum benchmark

In [ ]:
H, n_wires, symbols, coordinates, basis = build_hamiltonian(molecule)
Hmat = np.array(qml.matrix(H), dtype=float)
evals = np.sort(np.linalg.eigvalsh(Hmat))

print(f"Molecule: {molecule}")
print(f"Qubits:   {n_wires}")
print(f"Basis:    {basis}")
print(f"Lowest {max(10, num_states)} exact eigenvalues (Ha):")
for i, e in enumerate(evals[: max(10, num_states)]):
    print(f"#{i:>2}: {float(e): .10f}")

## Table summary helper

In [ ]:
def _summarize_ssvqe(result: dict, exact_evals: np.ndarray, *, label: str) -> None:
    energies = result["energies_per_state"]
    finals = [float(traj[-1]) for traj in energies]

    print("\n" + "=" * 80)
    print(label)
    print("=" * 80)

    for i, Ei in enumerate(finals):
        target = float(exact_evals[i]) if i < len(exact_evals) else float("nan")
        err = abs(Ei - target) if np.isfinite(target) else float("nan")
        print(f"State {i}: E_final = {Ei:+.10f} Ha   |ΔE_i| vs exact[{i}] = {err:.3e} Ha")

## 1) Noiseless k-state SSVQE

In [ ]:
noiseless_results = run_ssvqe(
    molecule=molecule,
    num_states=num_states,
    weights=weights,
    ansatz_name=ansatz_name,
    optimizer_name=optimizer_name,
    steps=steps,
    stepsize=stepsize,
    seed=seed,
    noisy=False,
    depolarizing_prob=0.0,
    amplitude_damping_prob=0.0,
    noise_model=None,
    plot=True,
    force=False,
)

_summarize_ssvqe(noiseless_results, evals, label="Noiseless SSVQE")

## 2) Noisy SSVQE using built-in depolarizing/amplitude probs

In [ ]:
depolarizing_prob = 0.05
amplitude_damping_prob = 0.05

noisy_results = run_ssvqe(
    molecule=molecule,
    num_states=num_states,
    weights=weights,
    ansatz_name=ansatz_name,
    optimizer_name=optimizer_name,
    steps=steps,
    stepsize=stepsize,
    seed=seed,
    noisy=True,
    depolarizing_prob=depolarizing_prob,
    amplitude_damping_prob=amplitude_damping_prob,
    noise_model=None,
    plot=True,
    force=False,
)

_summarize_ssvqe(
    noisy_results,
    evals,
    label=f"Noisy SSVQE: dep={depolarizing_prob}, amp={amplitude_damping_prob}",
)